#EDA

### Importaciones

In [ ]:
import pandas as pd
import json
import numpy as np
import pickle
import pyarrow.parquet as pq
import os
from textblob import TextBlob

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_archivo1 = '/content/drive/MyDrive/Colab Notebooks/yelp review business.csv'

# Cargar el archivo CSV en un DataFrame de Pandas
df = pd.read_csv(ruta_archivo1)

Como próximo paso, llevamos a cabo un análisis de sentimiento sobre las reseñas que los usuarios proporcionaron, con el objetivo de compararlo posteriormente con las calificaciones en estrellas que estos mismos usuarios otorgan al negocio. De esta manera, buscamos determinar cuál de estos criterios es más fidedigno para nuestra evaluación

In [ ]:
# Función para asignar categorías personalizadas
def categorizar_sentimiento(polaridad):
    if polaridad > 0.5:
        return 'Excelente'
    elif 0.1 <= polaridad <= 0.5:
        return 'Bueno'
    elif -0.1 <= polaridad < 0.1:
        return 'Regular'
    elif -0.5 <= polaridad < -0.1:
        return 'Malo'
    else:
        return 'Muy Malo'

# Aplica el análisis de sentimiento solo para reseñas presentes
mask = df['text'].notnull()
df.loc[mask, 'sentiment_analysis'] = df.loc[mask, 'text'].apply(lambda x: categorizar_sentimiento(TextBlob(str(x)).sentiment.polarity))

df.head()

,review_id,business_id,stars,text,date,name,address,postal_code,latitude,longitude,business_stars,review_count,categories,state,city,sentiment_analysis
0,pHwbdway4yeI-dSSmZA7-Q,PY9GRfzr4nTZeINf346QOw,4,We checked in around 2:30 pm. Check-in was qu...,2017-09-20 16:16:47,Peppermill Reno,2707 S Virginia St,89502.0,39.497687,-119.801139,4.0,2486,"Event Planning & Services, Casinos, Beauty & S...",Nevada,Reno,Bueno
1,JjzsXc2o_u0CbfKeJQPTNw,PY9GRfzr4nTZeINf346QOw,4,Loved the abvionce.. got pampered and relaxed....,2017-04-19 19:58:09,Peppermill Reno,2707 S Virginia St,89502.0,39.497687,-119.801139,4.0,2486,"Event Planning & Services, Casinos, Beauty & S...",Nevada,Reno,Bueno
2,JFjQ5RntPGhsNO2agAEYtg,PY9GRfzr4nTZeINf346QOw,5,Best casino in Reno. 4 deck and 2 deck black...,2005-05-15 00:39:29,Peppermill Reno,2707 S Virginia St,89502.0,39.497687,-119.801139,4.0,2486,"Event Planning & Services, Casinos, Beauty & S...",Nevada,Reno,Bueno
3,Tw3pUZn3SWspJuJIth0wgw,PY9GRfzr4nTZeINf346QOw,4,This place treats you right from the moment yo...,2007-05-27 07:09:22,Peppermill Reno,2707 S Virginia St,89502.0,39.497687,-119.801139,4.0,2486,"Event Planning & Services, Casinos, Beauty & S...",Nevada,Reno,Bueno
4,mTqx86waJjxil8Xl8Mwjcg,PY9GRfzr4nTZeINf346QOw,1,One of their guests went on a rampage and star...,2016-04-12 03:51:00,Peppermill Reno,2707 S Virginia St,89502.0,39.497687,-119.801139,4.0,2486,"Event Planning & Services, Casinos, Beauty & S...",Nevada,Reno,Regular


Categorizamos la columna "review_stars" asignando las etiquetas de 1 como "Muy Malo", 2 como "Malo", 3 como "Regular", y 4 y 5 como "Bueno" y "Excelente", respectivamente. Posteriormente, comparamos esta clasificación con nuestro análisis de sentimiento, donde aplicamos también una categorización. En este caso, asignamos las siguientes etiquetas: 0 representa un comentario negativo, 1 un comentario neutro y 2 un comentario positivo.

In [ ]:
# Define las categorías para las estrellas
star_mapping = {1: 'Muy Malo', 2: 'Malo', 3: 'Regular', 4: 'Bueno', 5: 'Excelente'}
df['stars_category'] = df['stars'].map(star_mapping)

In [ ]:
# Selecciona las columnas que deseas mostrar
selected_columns = ['stars_category', 'sentiment_analysis','stars','text']

# Muestra las primeras 20 filas de las columnas seleccionadas
df[selected_columns].head(5)

,stars_category,sentiment_analysis,stars,text
0,Bueno,Bueno,4,We checked in around 2:30 pm. Check-in was qu...
1,Bueno,Bueno,4,Loved the abvionce.. got pampered and relaxed....
2,Excelente,Bueno,5,Best casino in Reno. 4 deck and 2 deck black...
3,Bueno,Bueno,4,This place treats you right from the moment yo...
4,Muy Malo,Regular,1,One of their guests went on a rampage and star...


Como se observan rápidamente algunas discrepancias entre los dos análisis, creamos una tabla de contingencias para evaluarlas.

In [ ]:
# Crea la tabla de contingencia
contingency_table = pd.crosstab(df['sentiment_analysis'], df['stars_category'], margins=True, margins_name='Total')

# Renombra la columna 'All' a 'Total'
contingency_table.rename(columns={'All': 'Total'}, inplace=True)

# Visualiza la tabla de contingencia
print(contingency_table)

stars_category      Bueno  Excelente  Malo  Muy Malo  Regular  Total
sentiment_analysis                                                  
Bueno                7699      14789  1748      2006     3483  29725
Excelente             725       3653    38        46      118   4580
Malo                   45         52   494      2952      135   3678
Muy Malo                2          5    15       305        6    333
Regular               861        676  1912      4661     1265   9375
Total                9332      19175  4207      9970     5007  47691


Podemos observar que hay un total de 13416 coincidencias entre las dos columnas, de un total de 47691 filas. Se decide realizar una ponderación entre estas dos columnas para tomar en cuenta ambos análisis y llegar a un resultado más fidedigno.

se realiza un mapeo de las categorías de la columna "sentiment_analysis" a números del 1 al 5, asignándoles un valor numérico acorde a su grado de positividad. Luego, se asignan pesos a las métricas, especificando que la métrica de las estrellas de la revisión tiene un peso del 70%, mientras que el análisis de sentimiento tiene un peso del 30%. Finalmente, se calcula una nueva métrica combinada, donde se promedian ponderadamente los valores de las métricas de las estrellas y el análisis de sentimiento, obteniendo así una medida combinada que refleja tanto la evaluación de los usuarios como el análisis de sentimiento, considerando sus respectivos pesos. Este enfoque permite integrar ambas fuentes de información de manera equilibrada en una sola métrica para evaluar la satisfacción general

In [ ]:
# Mapeo de categorías a números del 1 al 5
sentiment_mapping_numerico = {'Excelente': 5, 'Bueno': 4, 'Regular': 3, 'Malo': 2, 'Muy Malo': 1}

# Aplicar el mapeo a la columna sentiment_analysis
df['sentiment_analysis_numerico'] = df['sentiment_analysis'].map(sentiment_mapping_numerico)

# Asignar pesos a las métricas
peso_review_stars = 0.7
peso_analisis_sentimiento = 0.3

# Calcular métrica combinada
df['metrica_combinada'] = (peso_review_stars * df['stars'] +
                           peso_analisis_sentimiento * df['sentiment_analysis_numerico']) / (peso_review_stars + peso_analisis_sentimiento)

se procede a mostrar el resultado de este analisis

In [ ]:
selected_columns = ['stars_category', 'sentiment_analysis','stars','metrica_combinada']

df[selected_columns].head(5)

,stars_category,sentiment_analysis,stars,metrica_combinada
0,Bueno,Bueno,4,4.0
1,Bueno,Bueno,4,4.0
2,Excelente,Bueno,5,4.7
3,Bueno,Bueno,4,4.0
4,Muy Malo,Regular,1,1.6


In [ ]:
columnas_a_eliminar = ['sentiment_analysis_numerico', 'stars_category', 'sentiment_analysis','review_count','text','stars']

df.drop(columnas_a_eliminar, axis=1, inplace=True)

In [ ]:
#renombramos la columna final
nombres_nuevos = {'metrica_combinada': 'review_stars'}

df.rename(columns=nombres_nuevos, inplace=True)

guardamos como csv con analisis de sentimientos realizado.

In [ ]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/yelp-review-business-sentiment.csv', index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47691 entries, 0 to 47690
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       47691 non-null  object 
 1   business_id     47691 non-null  object 
 2   date            47691 non-null  object 
 3   name            47691 non-null  object 
 4   address         46257 non-null  object 
 5   postal_code     47684 non-null  float64
 6   latitude        47691 non-null  float64
 7   longitude       47691 non-null  float64
 8   business_stars  47691 non-null  float64
 9   categories      47691 non-null  object 
 10  state           47691 non-null  object 
 11  city            47691 non-null  object 
 12  review_stars    47691 non-null  float64
dtypes: float64(5), object(8)
memory usage: 4.7+ MB


vamos a realizar una agrupacion de la tabla con nuestros reviews y nuestra metrica_combinada para llegar a una tabla final con negocion unicos y un analisis de sentimiento realizado sobre las reviews a estos negocios

In [ ]:
# Seleccionar las columnas deseadas
selected_columns = ['business_id', 'name', 'address', 'city', 'state', 'latitude', 'longitude', 'review_count', 'is_open', 'categories', 'metrica_combinada']

# Agrupar por 'business_id' y calcular el promedio de 'metrica_combinada'
df_grouped_business = df[selected_columns].groupby('business_id', as_index=False).agg({
    'metrica_combinada': 'mean',
    'name': 'first',
    'address': 'first',
    'city': 'first',
    'state': 'first',
    'latitude': 'first',
    'longitude': 'first',
    'review_count': 'first',
    'is_open': 'first',
    'categories': 'first',
})

# Renombrar la columna 'metrica_combinada' a 'stars'
df_grouped_business = df_grouped_business.rename(columns={'metrica_combinada': 'stars'})

# Visualizar el nuevo DataFrame agrupado
df_grouped_business.head(2)


,business_id,stars,name,address,city,state,latitude,longitude,review_count,is_open,categories
0,-12_gQ7NRcMWSRs97mRQdw,1.758333,Days Inn by Wyndham Gretna New Orleans,100 Westbank Expressway,Gretna,CA,29.910881,-90.051566,28,1,"Hotels & Travel, Hotels, Event Planning & Serv..."
1,-7kN5J6yiT3sZbsiDHquPA,4.642857,City Wide Limousine Service,6705 Governor Printz Blvd,Wilmington,AZ,39.779636,-75.478165,8,1,"Limos, Event Planning & Services, Hotels & Tra..."


una vez realizado procedemos a guardarlo y esta listo para consumirse

In [ ]:
df_grouped_business.to_csv('/content/drive/My Drive/Colab Notebooks/yelp-review-business-Final.csv', index=False)